# スクレイピングで取得したデータの加工

## 使い方
1. `requirements.txt` に記載のライブラリをインストール（緯度経度の取得、アプリ作成に必要なライブラリ含む）
2. [インポートとエクスポートの設定](#インポートとエクスポートの設定) を入力し `すべてを実行` ボタンで生成

## Note
- スクレイピングで取得したドリンクメニュー（ドリンク名およびドリンク価格）は、データ構造がリストの入れ子になっているため分割する必要がある
- 2022年12月16日現在、 `Pandas` で1度に分割できればいいのだが、難しそうだったのでドリンク名とドリンク価格で別々に分割し、スプレッドシート上で連結させている
- それぞれ分割したドリンク名の個数、ドリンク価格の個数が一致していて、分割されたドリンク名およびドリンク価格の店舗ごとのドリンクメニュー数も一致しているためデータにはズレは生じていないと思われる
  - ランダムで目視確認したところ問題なさそうだった

## 前準備

### パッケージのインポート

In [1]:
import pandas as pd
import numpy as np
import json
import datetime as dt
import jaconv # 全角を半角にするために必要

now = dt.datetime.now()
time = now.strftime("%y%m%d-%H%M")

### インポートとエクスポートの設定

### 読み込みするデータのファイル名、出力するデータのファイル名を変更する必要あり

- `import_file_name`: 読み込むJSONのファイル名を拡張子なしで入力
- `export_file_name`: 出力するファイル名を入力
  - インポートするデータに合わせて変更
  - 出力はCSV

In [2]:
# インポートするファイル名
import_file_name = "221218-1456_hotpepper_restaurant_lat_lng"
# import_file_name = "221218-1928_tabelog_restaurant_lat_lng"

# エクスポートするファイル名
export_file_name = "hotpepper"
# export_file_name = "tabelog"

### JSONの読み込み

In [3]:
# 店舗情報データJSONを読み込み
with open(f"export_files/{import_file_name}.json") as file:
    restaurant_list = json.load(file)

## `Pandas` でデータを整形する準備

### JSONから読み込んだデータをPandas形式に変換

In [4]:
df = pd.DataFrame(restaurant_list)

### ジャンルのリストを展開

In [5]:
df["restaurant_genre"] = df["restaurant_genre"].str.join("・")

# 以下、`ドリンク価格に全角が入っていた場合、半角に置換`の後に行うと思ったとおりのデータとならなかった
# JSONを読み込んですぐのこの場所ではうまくいく
# _restaurant_genre = df[["restaurant_genre"]]
# __restaurant_genre = _restaurant_genre["restaurant_genre"].str.join("・")
# __restaurant_genre

In [6]:
df

,restaurant_name,restaurant_address,restaurant_tel,restaurant_url,restaurant_genre,drink_name,drink_price,restaurant_latitude,restaurant_longitude
0,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,https://www.hotpepper.jp/strJ001292956/,居酒屋・和食,"[黒ラベル, アサヒスーパードライ（中瓶）/キリン一番搾り（中瓶）/赤星（中瓶）, オールフ...","[550, 660, 550, 462, 418, 462, 352, 418, 418, ...",34.702542,135.502122
1,榮華亭 梅田東通り店,大阪府大阪市北区堂山町１０－４ 忠兵衛ビル2・3・4F,050-5281-0957,https://www.hotpepper.jp/strJ000061620/,焼肉・ホルモン・韓国料理,"[生ビール, 瓶ビール, ノンアルコールビール アサヒドライゼロ, プレーン／レモン／ライム...","[400, 550, 380, 350, 350, 500, 400, 400, 750, ...",34.703090,135.504295
2,お初天神応援団 勝男,大阪府大阪市北区曽根崎２－７－１３,050-5452-6150,https://www.hotpepper.jp/strJ001259280/,居酒屋・和食,"[角ハイボール, トリスハイボール, マス男さん角ハイ, マス男さんトリハイ, 「名物」勝男...","[220, 165, 418, 308, 275, 328, 328, 328, 328, ...",34.699787,135.500493
3,最大165品 食べ放題 飲み放題 大衆居酒屋 梅田スタンド 梅田店,大阪府大阪市北区曽根崎２丁目 16-22 ジャスミン曽根崎ビル9F,050-5816-3131,https://www.hotpepper.jp/strJ001277115/,居酒屋,"[キリン一番搾り, シャンディーガフ, ハイボール, コークハイ, ジンジャーハイ, レモン...","[440, 462, 385, 440, 440, 418, 418, 418, 418, ...",34.701702,135.500410
4,個室 もつ鍋 泳ぎイカ 九州うまか 梅田店,大阪府大阪市北区角田町１－１ 地下１Ｆ,050-5834-2760,https://www.hotpepper.jp/strJ000973173/,居酒屋・創作料理,"[超炭酸ビームハイ, 超炭酸ビームハイ 濃いめ, 超炭酸ビームハイ MEGA, 柚子/巨峰/...","[380, 480, 660, 530, 430, 500, 750, 550, 650, ...",34.704002,135.501257
...,...,...,...,...,...,...,...,...,...
1350,BAR Chill,大阪府大阪市北区堂山町11-14 那由多ビル4F,050-5451-1923,https://www.hotpepper.jp/strJ001234421/,バー・カクテル,"[【シャーリーテンプル】, 【グレンフィディック】, 【アードベッグ】, 【マッカラン 12年】]","[-, -, -, -]",34.703695,135.504479
1351,個室均一居酒屋 きんいち お初天神店,大阪府大阪市北区曽根崎２-8-7 コーストスタジョーネ4F,06-6364-1111,https://www.hotpepper.jp/strJ000690098/,居酒屋,"[ジムビーム, 角ハイボール, ザ・プレミアムモルツ（中）, ジムビーム, 角瓶, 紀州南高...","[418, 418, 418, 418, 418, 418, 418, 418, 418, ...",34.699723,135.501128
1352,食べ放題専門居酒屋 日本一 梅田店,大阪府大阪市北区曽根崎２-8-7 コーストスタジョーネ4階,06-6364-3833,https://www.hotpepper.jp/strJ001029469/,居酒屋,"[ジムビーム, 角ハイボール, ザ・プレミアムモルツ（中）, ジムビーム, 角瓶, 紀州南高...","[418, 418, 418, 418, 418, 418, 418, 418, 418, ...",34.699723,135.501128
1353,カジュアルフレンチとワイン ポポンペット ラスール,大阪府大阪市北区角田町5-1 梅田楽天地ビル1Ｆ,06-6311-0570,https://www.hotpepper.jp/strJ001184483/,ダイニングバー・バル,"[バロン・アルベール・リュニヴェルセル・ブリュットNV ＊【平日は終日900円⇒600円】,...","[平日は終日600, 平日は終日800, 平日は終日1000, 16時まで550⇒400, ...",34.703865,135.500578


### リスト形式でカラムに格納されている `ドリンク名` と `ドリンク価格` を分割

- `ドリンク名` と `ドリンク価格` は、1つのカラムにリストで格納されている
- `ドリンク名` と `ドリンク価格` を同時に分割することがベストだと思うが、現在のスキルでは時間がかかりそうなのでそれぞれを行に分割した上でINDEXキーを軸に結合する選択を行った

#### `ドリンク名`に格納されているリストを分割

- ドリンク価格の列を削除し、ドリンクメニューはリスト配列になっているので行に分割
- 参考: [python \- How to unnest \(explode\) a column in a pandas DataFrame, into multiple rows \- Stack Overflow](https://stackoverflow.com/questions/53218931/how-to-unnest-explode-a-column-in-a-pandas-dataframe-into-multiple-rows)

In [7]:
del_drink_price = df.drop(columns="drink_price")

In [8]:
del_drink_price.drink_name = del_drink_price.drink_name.fillna({i: [] for i in del_drink_price.index})  # replace NaN with []

- ドリンク名を分割することで行が増えるため `ignore_index` を `True` に設定し、INDEX番号を振り直す（ドリンク価格のデータと結合するときにキーにできるのが `INDEX` しかないため必須）

In [9]:
drink_name_split = del_drink_price.explode("drink_name", ignore_index=True)

- `ドリンク名` と `ドリンク価格` を別々に分割するので行数が一致しているか確認

In [10]:
drink_name_split.shape

(50621, 8)

- CSVに出力する必要があればコメントアウトを解除

In [11]:
# drink_name_split.to_csv("{}_drink_name_split_{}.csv".format(export_file_name, time), index=False)

#### `ドリンク価格` に格納されているリストを分割

- ドリンク価格の列のみを `df_drink_price` に格納し、ドリンク価格のリストに格納されているデータを行に分割

In [12]:
df_drink_price = df[["drink_price"]]

In [13]:
df_drink_price = df_drink_price.drink_price.fillna({i: [] for i in df_drink_price.index})  # replace NaN with []

- ドリンク価格を分割することで行が増えるが、ドリンク名と違い `ignore_index` を `True` に設定しなくても自動的にINDEX番号が振り直される

In [14]:
drink_price_split = df_drink_price.explode("drink_price")

- `ドリンク名` と `ドリンク価格` を別々に分割するので行数が一致しているか確認

In [15]:
drink_price_split.shape

(50621,)

- CSVに出力する必要があればコメントアウトを解除

In [16]:
# drink_price_split.to_csv("/{}_drink_price_split_{}.csv".format(export_file_name, time), index=False)

### ドリンク名データとドリンク価格データの結合

In [17]:
# Indexを軸に結合
restaurant_new_list = pd.merge(drink_name_split, drink_price_split, how="outer", left_index=True, right_index=True)
# 結合したときにデータの行数に変化が出ていないか確認
restaurant_new_list.shape

(50621, 9)

In [18]:
restaurant_new_list.head(3)

,restaurant_name,restaurant_address,restaurant_tel,restaurant_url,restaurant_genre,drink_name,restaurant_latitude,restaurant_longitude,drink_price
0,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,https://www.hotpepper.jp/strJ001292956/,居酒屋・和食,黒ラベル,34.702542,135.502122,550
1,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,https://www.hotpepper.jp/strJ001292956/,居酒屋・和食,アサヒスーパードライ（中瓶）/キリン一番搾り（中瓶）/赤星（中瓶）,34.702542,135.502122,660
2,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,https://www.hotpepper.jp/strJ001292956/,居酒屋・和食,オールフリー（瓶）,34.702542,135.502122,550


### カラムを並び替える

- 店名、住所、緯度、経度、電話番号、URL、ドリンク名、ドリンク価格の順番に並び替える

In [19]:
restaurant_new_list = restaurant_new_list.reindex(columns=["drink_name", "drink_price", "restaurant_genre", "restaurant_name", "restaurant_address", "restaurant_tel", "restaurant_latitude", "restaurant_longitude", "restaurant_url"])
restaurant_new_list.head(3)

,drink_name,drink_price,restaurant_genre,restaurant_name,restaurant_address,restaurant_tel,restaurant_latitude,restaurant_longitude,restaurant_url
0,黒ラベル,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.502122,https://www.hotpepper.jp/strJ001292956/
1,アサヒスーパードライ（中瓶）/キリン一番搾り（中瓶）/赤星（中瓶）,660,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.502122,https://www.hotpepper.jp/strJ001292956/
2,オールフリー（瓶）,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.502122,https://www.hotpepper.jp/strJ001292956/


## アプリ用にデータの整形

### `ドリンク名`、`ドリンク価格`、`緯度`、`経度`の値が空の行を削除

#### ドリンク名およびドリンク価格が空の場合 `np.nan` に置換

- ドリンク名・ドリンク価格は、データがスクレイピングできないときには `""` を格納しているので `dropna` で削除できないため置換が必要
- 緯度・経度は `NaN` が入っているので置換する必要がない

In [20]:
# ドリンク名・ドリンク価格が `""` だったら `np.nan` に置換し、上書き
restaurant_new_list["drink_price"].replace("", np.nan, inplace=True)
restaurant_new_list["drink_name"].replace("", np.nan, inplace=True)
restaurant_new_list.head(3)

,drink_name,drink_price,restaurant_genre,restaurant_name,restaurant_address,restaurant_tel,restaurant_latitude,restaurant_longitude,restaurant_url
0,黒ラベル,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.502122,https://www.hotpepper.jp/strJ001292956/
1,アサヒスーパードライ（中瓶）/キリン一番搾り（中瓶）/赤星（中瓶）,660,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.502122,https://www.hotpepper.jp/strJ001292956/
2,オールフリー（瓶）,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.502122,https://www.hotpepper.jp/strJ001292956/


In [21]:
restaurant_new_list.shape

(50621, 9)

#### 欠損値がある行を CSV に出力

- 参考: 
  - [pandas 欠損値NaN処理一覧｜抽出、除去、補間 \- YutaKaのPython教室](https://www.yutaka-note.com/entry/pandas_na)
  - [PandasでNaNの列を処理する \- そうなんでげす](https://www.soudegesu.com/post/python/pandas_with_nan_columns/#%E5%90%84%E5%88%97%E3%81%ABnan%E3%81%8C%E5%AD%98%E5%9C%A8%E3%81%99%E3%82%8B%E3%81%8B%E3%82%92%E7%A2%BA%E8%AA%8D%E3%81%99%E3%82%8B)
  - [pandas 欠損値NaN処理一覧｜抽出、除去、補間 \- YutaKaのPython教室](https://www.yutaka-note.com/entry/pandas_na)

In [22]:
_drop_nan_list = restaurant_new_list.isna().any(axis=1)
drop_nan_list = restaurant_new_list.loc[_drop_nan_list, :]
drop_nan_list.shape

(779, 9)

In [23]:
# drop_nan_list.to_csv("export_files/{}_{}_drop_nan_list.csv".format(time, export_file_name), index=False)

#### 欠損値がある行を削除

In [24]:
restaurant_new_list.dropna(how="any", inplace=True)
# restaurant_new_list.head(3)

In [25]:
restaurant_new_list.shape

(49842, 9)

#### `ドリンク価格` が数値以外のデータを抽出

In [26]:
drop_price_list = restaurant_new_list[restaurant_new_list["drink_price"].apply(lambda s:pd.to_numeric(s, errors="coerce")).isnull()]
drop_price_list.shape

(4807, 9)

#### `ドリンク価格` が数値以外のデータを CSV に出力

In [27]:
# drop_price_list.to_csv("export_files/{}_{}_drop_price_list.csv".format(time, export_file_name), index=False)

#### `ドリンク価格` が数値以外のデータを削除

In [28]:
# `inplace=True` デフォルト値は `False` で `False` の場合は指定しなくても良い
# 削除前のデータを残しておく必要がないため `inplace=True` を設定して `restaurant_new_list` を上書きしている
restaurant_new_list.drop(restaurant_new_list[restaurant_new_list["drink_price"].apply(lambda s:pd.to_numeric(s, errors="coerce")).isnull()].index, inplace=True)

#### ドリンク価格に全角が入っていた場合、半角に変換

参考: [CSVデータをPythonでまとめて全角半角処理 \- Qiita](https://qiita.com/b2bmakers/items/19f5bb7a7f0c63a34493)

In [29]:
# jaconv をインストールしておく必要あり

def shori(column):
    list=  restaurant_new_list[column].values.tolist()
    new_list = []

    for li in list:
        li = jaconv.z2h(li,digit=True, ascii=True, kana=True)
        new_list.append(li)

    restaurant_new_list[column] = new_list

    return restaurant_new_list[column]

##処理したいしたいカラム名をリストに入れます。
columns = ["drink_price"]

#forで回します。
for column in columns:
    shori(column)

#strじゃないとエラーが出るときも。そんなときは
restaurant_new_list = restaurant_new_list.astype(str)

## アプリ用に整形後のデータをエクスポート

### カラム名をアプリで利用している名称に変更

In [30]:
restaurant_new_list.columns = ["menu", "price", "genre", "name", "adress", "tel", "lat", "lng", "url"]
restaurant_new_list.index = range(len(restaurant_new_list))
restaurant_new_list.head(3)

,menu,price,genre,name,adress,tel,lat,lng,url
0,黒ラベル,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
1,アサヒスーパードライ（中瓶）/キリン一番搾り（中瓶）/赤星（中瓶）,660,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
2,オールフリー（瓶）,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/


### 最終データを CSV に出力

In [31]:
restaurant_new_list.to_csv("export_files/{}_{}_restaurant.csv".format(time, export_file_name), index=False)

# ここからデータクレンジングの検討 @2022/12/17

In [32]:
restaurant_new_list

,menu,price,genre,name,adress,tel,lat,lng,url
0,黒ラベル,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
1,アサヒスーパードライ（中瓶）/キリン一番搾り（中瓶）/赤星（中瓶）,660,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
2,オールフリー（瓶）,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
3,シャンディーガフ/レッドアイ,462,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
4,ハイボール（デュワーズ）/ハイボール（角）,418,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
...,...,...,...,...,...,...,...,...,...
45030,ワイン用びぶどうで造ったぶどうジュース（赤・白）,528,ダイニングバー・バル,カジュアルフレンチとワイン ポポンペット ラスール,大阪府大阪市北区角田町5-1 梅田楽天地ビル1Ｆ,06-6311-0570,34.7038649,135.5005779,https://www.hotpepper.jp/strJ001184483/
45031,ピーチネクター,418,ダイニングバー・バル,カジュアルフレンチとワイン ポポンペット ラスール,大阪府大阪市北区角田町5-1 梅田楽天地ビル1Ｆ,06-6311-0570,34.7038649,135.5005779,https://www.hotpepper.jp/strJ001184483/
45032,ジンジャーエール,418,ダイニングバー・バル,カジュアルフレンチとワイン ポポンペット ラスール,大阪府大阪市北区角田町5-1 梅田楽天地ビル1Ｆ,06-6311-0570,34.7038649,135.5005779,https://www.hotpepper.jp/strJ001184483/
45033,オレンジジュース,418,ダイニングバー・バル,カジュアルフレンチとワイン ポポンペット ラスール,大阪府大阪市北区角田町5-1 梅田楽天地ビル1Ｆ,06-6311-0570,34.7038649,135.5005779,https://www.hotpepper.jp/strJ001184483/


## ビールを抽出

In [33]:
def shori(column):
    list=  restaurant_new_list[column].values.tolist()
    new_list = []

    for li in list:
        li = jaconv.z2h(li, digit=True, ascii=True, kana=False)
        new_list.append(li)

    restaurant_new_list[column] = new_list

    return restaurant_new_list[column]

##処理したいしたいカラム名をリストに入れます。
columns = ["menu"]

#forで回します。
for column in columns:
    shori(column)

#strじゃないとエラーが出るときも。そんなときは
restaurant_new_list = restaurant_new_list.astype(str)

In [34]:
_restaurant_beer_list = restaurant_new_list[restaurant_new_list.menu.str.contains("アサヒ|エビス|ヱビス|オリオンビール|ギネス|キリン|サッポロ|サントリー|スーパードライ|ニッポンプレミアム|ハートランドビール|ハイネケン|バドワイザー|ヒューガルデン|モルツ|ラガー|一番搾り生ビール|黒ラベル|生黒|生搾り|青島ビール|コロナ")]
_restaurant_beer_list.shape

(2206, 9)

In [35]:
_restaurant_beer_list_drop = _restaurant_beer_list[~_restaurant_beer_list["menu"].str.contains("FREE|OLD|ROYAL|V.S.O.P|VOGA|VSOP|XO|ZERO|アペロール|ウィスキー|ウーロン茶|サントリーオールド|キリンフリー|キリンレモン|ネーブル|グレープフルーツ|オレンジ|サワー|サングリア|ゼロ|ドライゼロ|ノンアルコール|ハイボール|ビアリー|ファーム|フリー|マッコリ|ライム|リザーブ|レモン|ロイヤル|ローヤル|ワイナリー|烏龍茶|角|響|山崎|翠|知多|酎ハイ|梅酒|白州|碧|余市|零|吟醸|純米")]
_restaurant_beer_list_drop["menu"] = _restaurant_beer_list_drop["menu"].str.strip()
_restaurant_beer_list_drop.shape

/tmp/ipykernel_3976/2455202692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _restaurant_beer_list_drop["menu"] = _restaurant_beer_list_drop["menu"].str.strip()


(1726, 9)

In [36]:
_restaurant_beer_list_drop["menu"] = _restaurant_beer_list_drop["menu"].str.replace("エビス", "ヱビス")


/tmp/ipykernel_3976/3008801843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _restaurant_beer_list_drop["menu"] = _restaurant_beer_list_drop["menu"].str.replace("エビス", "ヱビス")


In [37]:
_restaurant_beer_list_drop

,menu,price,genre,name,adress,tel,lat,lng,url
0,黒ラベル,550,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
1,アサヒスーパードライ(中瓶)/キリン一番搾り(中瓶)/赤星(中瓶),660,居酒屋・和食,海鮮酒場 うおぷく 梅田店,大阪府大阪市北区堂山町4-12 白馬車ビル３F,050-5861-6095,34.702542,135.5021224,https://www.hotpepper.jp/strJ001292956/
135,サントリー ザ・プレミアムモルツ 中,198,居酒屋・和食,お初天神応援団 勝男,大阪府大阪市北区曽根崎２－７－１３,050-5452-6150,34.6997866,135.500493,https://www.hotpepper.jp/strJ001259280/
136,サントリー ザ・プレミアムモルツ お得ジョッキ,286,居酒屋・和食,お初天神応援団 勝男,大阪府大阪市北区曽根崎２－７－１３,050-5452-6150,34.6997866,135.500493,https://www.hotpepper.jp/strJ001259280/
166,キリン一番搾り,440,居酒屋,最大165品 食べ放題 飲み放題 大衆居酒屋 梅田スタンド 梅田店,大阪府大阪市北区曽根崎２丁目 16-22 ジャスミン曽根崎ビル9F,050-5816-3131,34.7017024,135.5004102,https://www.hotpepper.jp/strJ001277115/
...,...,...,...,...,...,...,...,...,...
44860,スーパードライ(中瓶),550,居酒屋,鶴すし 梅田お初天神店,大阪府大阪市北区曽根崎２-10-14,050-5831-0747,34.7009597,135.5007968,https://www.hotpepper.jp/strJ001281847/
44899,アサヒスーパードライ 瓶ビール(中),638,居酒屋,新世界串カツ いっとく 大阪駅前第４ビル店,大阪府大阪市北区梅田１－１１－４ 大阪駅前第4ビル 地下2階,06-6348-9499,34.6998973,135.4988268,https://www.hotpepper.jp/strJ001153979/
44931,ザ・プレミアムモルツ(中),418,居酒屋,個室均一居酒屋 きんいち お初天神店,大阪府大阪市北区曽根崎２-8-7 コーストスタジョーネ4F,06-6364-1111,34.6997226,135.5011282,https://www.hotpepper.jp/strJ000690098/
44953,ザ・プレミアムモルツ(中),418,居酒屋,食べ放題専門居酒屋 日本一 梅田店,大阪府大阪市北区曽根崎２-8-7 コーストスタジョーネ4階,06-6364-3833,34.6997226,135.5011282,https://www.hotpepper.jp/strJ001029469/


In [38]:
_restaurant_beer_list_drop = _restaurant_beer_list_drop.sort_values(by="menu", ascending=True)

In [39]:
# print(_restaurant_beer_list_drop['menu'].dtypes)
# _restaurant_beer_list_drop["menu"].astype(str)

In [40]:
_restaurant_beer_list_drop.to_csv("export_files/{}_{}_beer.csv".format(time, export_file_name), index=False)